In [1]:
import pandas as pd
import numpy as np
from scipy.stats import gamma, norm
from pathlib import Path

# -----------------------------
# Load Data
# -----------------------------
file_path = Path(r"D:\climate change\yearly_averages.xlsx")
sheet_name = "Rainfall_yearly"

# Read without skipping rows first
df_raw = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

# Find the first row that contains '1975' or any year-like value
year_row_idx = None
for i, row in df_raw.iterrows():
    if row.astype(str).str.contains("1975").any():
        year_row_idx = i
        break

# Now read from that row properly
df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=year_row_idx)
df.rename(columns={df.columns[0]: "Year"}, inplace=True)

# Convert Year column
df["Year"] = pd.to_datetime(df["Year"], errors="coerce").dt.year
df = df.set_index("Year").sort_index()
df = df.apply(pd.to_numeric, errors="coerce")

print("✅ Data loaded successfully!")
print(df.head())

# -----------------------------
# Helper functions
# -----------------------------
def compute_spi(rain_series, alpha, beta):
    spi = []
    for x in rain_series:
        if np.isnan(x):
            spi.append(np.nan)
        elif x == 0:
            spi.append(norm.ppf(0.0001))
        else:
            prob = gamma.cdf(x, alpha, scale=beta)
            spi.append(norm.ppf(prob))
    return pd.Series(spi, index=rain_series.index)

def estimate_beta_fixed_alpha(data, alpha):
    data = data[data > 0]
    return np.mean(data) / alpha if len(data) > 0 else np.nan

def classify_spi(spi_val):
    if pd.isna(spi_val):
        return np.nan
    elif spi_val <= -2.0:
        return "Extremely Dry"
    elif spi_val <= -1.5:
        return "Severely Dry"
    elif spi_val <= -1.0:
        return "Moderately Dry"
    elif spi_val < 1.0:
        return "Near Normal"
    elif spi_val < 1.5:
        return "Moderately Wet"
    elif spi_val < 2.0:
        return "Severely Wet"
    else:
        return "Extremely Wet"

# -----------------------------
# Periods
# -----------------------------
period1 = (1975, 1995)
period2 = (1996, 2020)

spi_results = {}
spi_class_results = {}
param_results = []

for station in df.columns:
    rain = df[station].dropna()
    part1 = rain[(rain.index >= period1[0]) & (rain.index <= period1[1])]
    part2 = rain[(rain.index >= period2[0]) & (rain.index <= period2[1])]

    if len(part1) < 5 or len(part2) < 5:
        print(f"⚠️ Skipping {station}: insufficient data ({len(part1)} / {len(part2)})")
        continue

    # --- Stage 1 ---
    rain1 = part1[part1 > 0]
    alpha1, loc1, beta1 = gamma.fit(rain1, floc=0)
    spi1 = compute_spi(part1, alpha1, beta1)
    spi1_class = spi1.apply(classify_spi)

    # --- Stage 2 ---
    rain2 = part2[part2 > 0]
    beta2 = estimate_beta_fixed_alpha(rain2, alpha1)
    spi2 = compute_spi(part2, alpha1, beta2)
    spi2_class = spi2.apply(classify_spi)

    spi_results[f"{station}_SPI_1975_1995"] = spi1
    spi_results[f"{station}_SPI_1996_2020"] = spi2
    spi_class_results[f"{station}_Class_1975_1995"] = spi1_class
    spi_class_results[f"{station}_Class_1996_2020"] = spi2_class

    param_results.append({
        "Station": station,
        "Alpha_1975_1995": round(alpha1, 4),
        "Beta_1975_1995": round(beta1, 4),
        "Alpha_1996_2020": round(alpha1, 4),
        "Beta_1996_2020": round(beta2, 4)
    })

# -----------------------------
# Save Outputs
# -----------------------------
output_dir = Path(r"D:\climate change")
output_dir.mkdir(exist_ok=True)

spi_df = pd.DataFrame(spi_results)
spi_class_df = pd.DataFrame(spi_class_results)
param_df = pd.DataFrame(param_results)

spi_df.to_csv(output_dir / "SPI_Values_1975_2020.csv", float_format="%.2f")
spi_class_df.to_csv(output_dir / "SPI_Drought_Classes_1975_2020.csv", index=True)
param_df.to_csv(output_dir / "SPI_Parameters_1975_2020.csv", index=False)

print("\n✅ SPI computation completed successfully!")
print(f"📄 SPI values saved at: {output_dir / 'SPI_Values_1975_2020.csv'}")
print(f"📄 Drought classes saved at: {output_dir / 'SPI_Drought_Classes_1975_2020.csv'}")
print(f"📄 Parameters saved at: {output_dir / 'SPI_Parameters_1975_2020.csv'}")


✅ Data loaded successfully!
      13.746082  5.581068   3.341288   1.646904   3.639041   7.264932   \
Year                                                                     
1976  10.261421   4.979317   4.424235   1.152678   4.411448  15.148279   
1977   9.820521   3.722274   1.694658   1.202740   2.532329  13.225836   
1978  10.500110   4.584438   2.931726   1.551863   4.070027  17.594411   
1979  15.060137   5.728767   1.818575   1.770274   4.585178  33.800027   
1980   9.728115   4.636776   2.298552   1.763716   4.613934  19.423388   

      3.325014   3.557836   1.629890   1.291041   ...  2.096795   2.129041   \
Year                                              ...                         
1976   6.029699   5.054863   1.856612   1.036448  ...   1.624536   4.234153   
1977   4.467753   4.865781   1.074904   0.446822  ...   1.503808   1.738493   
1978   6.220219   4.705890   1.622849   0.960904  ...   2.248027   3.130000   
1979   6.022082   6.692849   0.797753   0.643644  ...   2.